In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [31]:
# Definir la región de interés
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/aoi_pampa')
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

In [32]:
roi_feature = ee.FeatureCollection(roi).first()
roi_geometry = roi_feature.geometry()
roi_coordinates = roi_geometry.coordinates().getInfo()


# Guardar las coordenadas en una variable
roi_coords = roi_coordinates
ee_roi =  ee.Geometry.Polygon(roi_coords) 

In [33]:
# Definir el año de análisis
year = 2020

# Definir la fecha de inicio y fin para filtrar la colección
start = f'{year}-01-01'
end = f'{year}-12-31'

In [34]:
# Cargar la colección de Landsat 7
l7_col = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

def mask_clouds_landsat(image):
    # Seleccionar la banda de máscara de calidad (QA_PIXEL)
    qa = image.select('QA_PIXEL')

    # Crear una máscara para las nubes y sombras de nubes
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()

    # Aplica las máscaras de nubes y sombras
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))

    # Aplicar la máscara de nubes y sombras
    return image.updateMask(mask)

# Aplicar la máscara de nubes a la colección de Landsat 7
l7_masked = l7_col.map(mask_clouds_landsat)
image = l7_masked.median().toFloat().clip(roi)

# Calcular índices
ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
mndwi = image.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')
ndbi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDBI')

evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': image.select('SR_B4'),
        'RED': image.select('SR_B3'),
        'BLUE': image.select('SR_B1')
    }
).rename('EVI')

savi = image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': image.select('SR_B4'),
        'RED': image.select('SR_B3')
    }
).rename('SAVI')

image = image.addBands([ndvi, mndwi, ndbi, evi, savi])

# Cargar la colección Dynamic World y filtrar por la grilla y la fecha
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-01')

# Añadir la banda de etiqueta de Dynamic World a la imagen
dw_label = dw_col.select('label').median().clip(roi)
image = image.addBands(dw_label.rename('label'))

dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)
aspect = ee.Terrain.aspect(dem).rename('aspect').clip(roi)

image = image.addBands(dem.rename('elevation'))
image = image.addBands(slope)
image = image.addBands(aspect)

import math

def terrain_correction_palsar(image, roi, angle_image):
    # Usar la ROI para recortar el SRTM
    srtm = ee.Image('USGS/SRTMGL1_003').clip(roi)
    
    # Convertir la imagen a dB usando la fórmula γ₀ = 10 log₁₀(DN²) - 83.0 dB

    dn_squared = image.pow(2)   
    backscatter = ee.Image.constant(10).multiply(dn_squared.log10()).subtract(83.0)
    theta_i = angle_image  
    
    # Obtener el aspecto promedio de la región de interés
    phi_i = ee.Terrain.aspect(theta_i).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,  # Usar la ROI
        scale=1000
    ).get('aspect')
    
    # Convertir phi_i a una imagen constante
    phi_i_image = ee.Image.constant(phi_i)
    
    # Geometría del terreno
    alpha_s = ee.Terrain.slope(srtm).select('slope')
    phi_s = ee.Terrain.aspect(srtm).select('aspect')
    
    # Geometría del modelo
    phi_r = phi_i_image.subtract(phi_s)
    
    # Convertir todo a radianes
    phi_r_rad = phi_r.multiply(math.pi / 180)
    alpha_s_rad = alpha_s.multiply(math.pi / 180)
    theta_i_rad = theta_i.multiply(math.pi / 180)
    ninety_rad = ee.Image.constant(90).multiply(math.pi / 180)

    # Pendiente en el rango
    alpha_r = (alpha_s_rad.tan().multiply(phi_r_rad.cos())).atan()

    # Pendiente en acimut
    alpha_az = (alpha_s_rad.tan().multiply(phi_r_rad.sin())).atan()

    # Ángulo de incidencia local
    theta_lia = (alpha_az.cos().multiply((theta_i_rad.subtract(alpha_r)).cos())).acos()
    theta_lia_deg = theta_lia.multiply(180 / math.pi)

    # Gamma_nought_flat usando la fórmula ajustada
    gamma0 = backscatter.divide(theta_i_rad.cos())
    gamma0_dB = gamma0.rename('gamma0_dB')

    # Modelo volumétrico
    nominator = (ninety_rad.subtract(theta_i_rad).add(alpha_r)).tan()
    denominator = (ninety_rad.subtract(theta_i_rad)).tan()
    vol_model = (nominator.divide(denominator)).abs()

    # Aplicar el modelo
    gamma0_volume = gamma0.divide(vol_model)
    gamma0_volume_dB = gamma0_volume.rename('gamma0_volume_dB')

    return gamma0_dB


def preprocess(image, roi, angle_image):
    image = terrain_correction_palsar(image, roi, angle_image)
    speckle_filtered = image.focal_median(kernelType='circle', radius=50, units='meters')
    return speckle_filtered


angle = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('angle')  
    .median() 
)

img_hh = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HH')  # Seleccionar solo HH
    .map(lambda img: preprocess(img, roi, angle))  # Pasar roi y angle_image al preprocesamiento
)

img_hv = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HV')  # Seleccionar solo HV
    .map(lambda img: preprocess(img, roi, angle))  # Pasar roi y angle_image al preprocesamiento
)

palsar_hh_median = img_hh.median().rename('HH_dB')
palsar_hv_median = img_hv.median().rename('HV_dB')

hh_plus_hv = palsar_hh_median.add(palsar_hv_median).rename('HH_plus_HV')
hh_minus_hv = palsar_hh_median.subtract(palsar_hv_median).rename('HH_minus_HV')
hh_div_hv = palsar_hh_median.divide(palsar_hv_median).rename('HH_div_HV')
rvi = palsar_hv_median.divide(palsar_hh_median.add(palsar_hv_median)).multiply(4).rename('RVI_palsar')

palsar_combined = ee.Image.cat([palsar_hh_median, palsar_hv_median, hh_plus_hv, hh_minus_hv, hh_div_hv, rvi]).clip(roi)
image = image.addBands(palsar_combined)


In [35]:
def calculate_texture_metrics(image, band_name, roi, size=5):
    # Convertir la banda a tipo int32 (no se realiza normalización adicional)
    band_int = image.select(band_name).toInt32().clip(roi)
    
    # Calcular texturas usando la banda convertida a int32
    glcm = band_int.glcmTexture(size=size)
     
    contrast = glcm.select(f'{band_name}_contrast').rename(f'{band_name}_Contrast')
    correlation = glcm.select(f'{band_name}_corr').rename(f'{band_name}_Correlation')
    entropy = glcm.select(f'{band_name}_ent').rename(f'{band_name}_Entropy')
    inertia = glcm.select(f'{band_name}_inertia').rename(f'{band_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])
image = calculate_texture_metrics(image, 'HH_dB', roi)
image = calculate_texture_metrics(image, 'HV_dB', roi)

bands =  ['NDBI','MNDWI','HV_dB','SR_B4','SR_B3','elevation','slope','EVI','HV_dB_Entropy','HV_dB_Correlation','SR_B2','HH_dB_Correlation','SR_B1','aspect']
 

image = image.select(bands)

In [36]:
Map.centerObject(roi, 10)
Map.addLayer(image, {}, 'FAPAR', False)


Map

Map(center=[-32.40021449709678, -58.31767356390013], controls=(WidgetControl(options=['position', 'transparent…

In [37]:
feature_names =  ['NDBI','MNDWI','HV_dB','SR_B4','SR_B3','elevation','slope','EVI','HV_dB_Entropy','HV_dB_Correlation','SR_B2','HH_dB_Correlation','SR_B1','aspect']
 

label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/Rf_uruguay"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [27]:
# feature_names =  ['NDBI','MNDWI','HV_dB','SR_B4','SR_B3','elevation','slope','EVI','HV_dB_Entropy','HV_dB_Correlation','SR_B2','HH_dB_Correlation','SR_B1','aspect']


# label = "agbd"
# user_id = 'users/facuboladerasgee'
# def recombine_model_parts(user_id, base_asset_id, num_parts):
#     parts = [ee.FeatureCollection(f"{base_asset_id}_part_{i}") for i in range(num_parts)]
#     combined_fc = ee.FeatureCollection(parts[0])
    
#     for part in parts[1:]:
#         combined_fc = combined_fc.merge(part)
    
#     return ml.fc_to_classifier(combined_fc)

# # Ejemplo de uso:
# num_parts = 2  # Ajusta este número según la cantidad de partes exportadas
# base_asset_id = f"{user_id}/Rf_Nepal_corregido"
# recombined_classifier = recombine_model_parts(user_id, base_asset_id, num_parts)

# # Clasificar la imagen con el clasificador recombinado
# classified = image.select(feature_names).classify(recombined_classifier)


In [14]:
# # Reducir la imagen clasificada a su valor máximo en la región de interés
# max_value = classified.reduceRegion(
#     reducer=ee.Reducer.max(),
#     geometry=roi,
#     scale=100,
#     maxPixels=1e9
# )

# # Obtener el valor máximo de la reducción
# max_agbd = max_value.get('classification').getInfo()
# print(f"Max value of the classified image: {max_agbd}")

Max value of the classified image: 155.93490694999983


In [38]:
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.centerObject(roi)

visPredictedBiomass = {
    'min': 0,
    'max': 200,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}

Map = geemap.Map(zoom=11)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

Map.addLayer(
    classified,
    visPredictedBiomass,
    "classification",)
Map.add_colorbar(visPredictedBiomass, label="Predicted Biomass")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [39]:
import time

# Parámetros de exportación
task = ee.batch.Export.image.toDrive(
    image=classified,
    description='costa_uruguay',
    folder='Estimators_remote_sensing_pampa',
    region=ee_roi,
    scale=30,
    crs='EPSG:4326',  # CRS adecuado para Nepal
    
    
)
task.start()

print("Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.")

# Esperar a que la tarea de exportación se complete (opcional)
while task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

if task.status()['state'] == 'COMPLETED':
    print('Exportación completada con éxito.')
else:
    print('Error en la exportación:', task.status())

Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación completada con éxito.
